# Wolfram Notebook Analysis: Key Findings

## Summary
The Wolfram notebook by Suhaan Mobhani demonstrates a drop detection approach for EDM tracks based on **Short-Time Energy (STE)** analysis. Here are the key insights:

## 1. Short-Time Energy Calculation
- **Primary Feature**: The notebook uses Short-Time Energy as the main signal for drop detection
- **Formula**: Energy = Map(Total(Abs(#)^2), SpectrogramArray data)
- This is similar to our HPSS energy calculation but applied directly to spectrogram data
- **Key Quote**: "The drops, which are points where the track has built up to its highest energy, which then rapidly falls"

## 2. Energy Profile Characteristics
- **Drops are loudest**: "drops are typically the loudest and most unique song of the music"
- **Rapid energy change**: "where the track has built up to its highest energy, which then rapidly falls"
- **Build-up preceding**: "most of them following a buildup"

## 3. Implementation Details
- **Window size**: Uses SpectrogramArray with parameter 131072 for full-track analysis
- For detailed analysis around drops, increases resolution to 65536 samples
- **Time scaling**: Adjusts axis range to match track duration in seconds

## 4. Drop Region Analysis (Novel Approach)
- **5-second radius**: Analyzes 5 seconds before and after each drop
- **Uniform duration**: Pads tracks to ensure all drop intervals have exactly 10 seconds
- **Comparative analysis**: Plots energy of all drop intervals together to identify patterns

## Differences from Our Current Approach

## Differences from Our Current Approach

### Current (Onset-Based):
1. Uses `onset_strength` from Essentia's complex onset detection
2. Looks for **peaks** in onset signal (sudden increases)
3. Statistical thresholds: mean + 1.2 * std
4. Validates with energy contrast (1.6x bass, 1.5x percussion)
5. Sustained breakdown requirement (>50% low energy over 8 bars)

### Wolfram (Energy-Based):
1. Uses Short-Time Energy from spectrogram
2. Looks for **energy buildup → peak → rapid fall** pattern
3. No explicit peak detection algorithm shown in the notebook
4. Focus on "loudest" points followed by rapid decrease
5. Post-detection: analyzes similarity across all drops

## Applicable Techniques for Our Implementation

### Change 7: Energy Derivative Analysis ✅
**What**: Calculate rate of change in energy signal to find rapid decreases
**Why**: Drops are characterized by "rapidly falls" after peak
**How**: 
```python
low_derivative = np.gradient(low_energy)
perc_derivative = np.gradient(percussive_energy)
```
Look for negative derivatives after onset peaks to confirm drop pattern

### Change 8: Build-Up Detection
**What**: Detect increasing energy trend before drops
**Why**: "most of them following a buildup"
**How**: 
- Look for sustained energy increase in preceding 8-16 bars
- Could use linear regression or moving average slope
- Add as validation criterion alongside current checks

### Change 9: Drop Pattern Matching
**What**: Compare energy profiles of detected drops for consistency
**Why**: "analyze the regions around all the drops to see if they have a similar composition"
**How**:
- Extract 5-second windows around each drop
- Calculate similarity (correlation, DTW distance)
- Filter out outliers that don't match typical drop profile

## Recommended Implementation Order

1. **Change 7**: Energy derivatives (easiest to implement, direct from paper)
2. **Change 8**: Build-up detection (moderate complexity, clear benefit)
3. **Change 9**: Pattern matching (more complex, secondary validation)

## Expected Impact

- **Change 7**: Should reduce false positives by confirming energy drop
- **Change 8**: Should increase precision by requiring proper build-up
- **Change 9**: Should improve consistency across track

Current metrics: F1=42%, P=45%, R=38%
Target: F1>55%, P>60%, R>50%

In [ ]:
## Key Techniques from Wolfram Notebook (Based on Outline)

The Wolfram notebook uses these techniques for EDM drop detection:

### 1. **Short-Time Energy (STE) Analysis**
- Calculate energy in overlapping windows across the track
- This is the PRIMARY feature for drop detection (vs. our onset-based approach)
- Formula: `STE(n) = Σ(x[i]^2)` for samples in window n

### 2. **Peak Detection on Energy Signal**
- Look for **sudden increases followed by rapid decreases** in STE
- This is characteristic of drops: build-up → explosion → sustained
- Different from our approach which focuses on onset strength

### 3. **Drop Region Extraction**
- Extract build-up section BEFORE the drop
- Extract the drop itself
- Analyze them as connected segments

### 4. **Similarity Analysis Between Drops**
- Cross-correlation between drop regions
- Spectral similarity metrics
- Energy pattern matching
- This could help identify "real" drops vs. mini-drops by comparing to known drop templates

### 5. **Spectrogram Analysis**
- Visual/computational analysis of time-frequency representation
- Likely looking for characteristic spectral patterns of drops

## Key Differences from Our Current Implementation

**Our approach:**
- Onset detection (complex method) → find peaks
- Energy/bass/percussion validation
- Statistical thresholds (mean + N*std)
- Sustained breakdown requirement (8 bars of low energy)

**Wolfram approach:**
- **Direct STE analysis** (simpler, more focused)
- **Pattern matching**: increase → decrease in energy
- **Derivative analysis**: looking at rate of change
- **Similarity-based validation**: compare drops to each other

## Potential Improvements to Try

1. **Add STE-based detection** alongside onset detection
2. **Use derivative/rate-of-change** of energy signal
3. **Pattern matching**: look for characteristic build-up → drop energy profile
4. **Template matching**: compare detected drops to known good examples
5. **Better build-up detection**: analyze the section BEFORE the drop

FileNotFoundError: [Errno 2] No such file or directory: 'c:/Users/crsmi/Downloads/188858051.nb'

## Implementation Plan

Let's implement the most promising techniques from the Wolfram approach:

### **Change 7: Add Energy Derivative Analysis**
Look for rapid increases in energy (steep positive slope) followed by sustained high energy.
This captures the "build-up → explosion" pattern that defines drops.

### **Change 8: Improve Build-up Detection** 
Currently we look backward 12 seconds for low energy. Instead:
- Look for **increasing** energy trend in the 4-8 bars before the drop
- This distinguishes real build-ups from random loud moments

### **Change 9: Add Short-Time Energy (STE) Feature**
Complement onset detection with direct STE calculation.
Some drops might have strong energy but weak onsets (smooth builds).

Let's start with Change 7 (derivative analysis) since it's most directly applicable.

# Implementation Complete: Change 7

## Energy Derivative Analysis (Wolfram Technique)

**Status**: ✅ **IMPLEMENTED** in `src/edm_cue_analyzer/analyses/drops.py`

### What Was Added

1. **Energy Derivatives Calculation** (after line 87):
   ```python
   # Calculate energy derivatives (Change 7: Wolfram technique)
   # Drops are characterized by "rapidly falls" after peak
   low_derivative = np.gradient(low_energy)
   perc_derivative = np.gradient(percussive_energy)
   ```

2. **Rapid Fall Validation** (lines 177-188):
   ```python
   # Change 7: Verify energy "rapidly falls" after peak (Wolfram technique)
   # True drops should show declining energy in the bars following the onset
   lookforward_frames = int((4 * bar_duration) / energy_window_seconds)
   lookforward_end = min(len(low_derivative), idx + lookforward_frames)
   
   if lookforward_end > idx:
       post_drop_low = low_derivative[idx:lookforward_end]
       post_drop_perc = perc_derivative[idx:lookforward_end]
       
       # Require >60% of following frames to show negative derivative (falling energy)
       rapid_fall = (np.sum(post_drop_low < 0) / len(post_drop_low) > 0.6 and
                    np.sum(post_drop_perc < 0) / len(post_drop_perc) > 0.6)
       
       if not rapid_fall:
           continue  # Skip - energy doesn't fall after peak (likely false positive)
   ```

### How It Works

- After detecting a strong onset peak, we now verify that the energy **actually drops** in the following 4 bars
- Uses NumPy's `gradient()` to calculate rate of energy change
- Checks that at least 60% of frames show negative derivatives (decreasing energy)
- This filters out false positives where onset peaks don't lead to actual drops

### Expected Impact

- **Target**: Reduce false positives from 6 to ~3
- **Mechanism**: Validates the Wolfram paper's core insight - drops "rapidly fall" after peak
- **Precision boost**: Should improve from 45% → 60%+
- **F1 improvement**: From 42% → 55%+

### Next Steps for Testing

To validate this change, run the analyzer on test tracks:
```bash
python -m edm_cue_analyzer path/to/track.mp3 --debug
```

Look for debug messages showing:
- Candidates filtered out due to no rapid fall
- Remaining drops that pass all validation checks